In [1]:
import sys, os; sys.path.append(os.path.abspath("../"));

from importlib import reload
import pandas as pd, numpy as np, torch
import _settings
import utils.utils as utils
import data.dataloader as dld; reload(dld)
import models.regmodel as regmodel; reload(regmodel)
import models.conformal as conformal; reload(conformal)
import demos.demo as demo; reload(demo)
import demos.regression as reg; reload(reg)
import demos.experiments as exp; reload(exp)
import utils.eval_utils as eval_utils; reload(eval_utils)
import matplotlib.pyplot as plt

DATA_PATH = _settings.DATA_PATH
CACHE_PATH = os.path.join(_settings.WORKSPACE, 'Baselines')

In [2]:
NSEEDS = 10
default_fitkwargs = exp.get_default_fitkwargs()

METHOD_NAME = 'LVD'
conf_baselines = ['MADSplit', 'CQR']
baselines = ["DJ", "DE", "MCDP", "PBP"]
use_methods = [METHOD_NAME] + conf_baselines + baselines
all_methods = [METHOD_NAME] + conf_baselines + baselines
width_methods = [METHOD_NAME] + conf_baselines

method_rename = {"LVD":"\\methodname", "DE_Correct": "DE"}

In [3]:
#Define datasets to look at
small_datasets = [_settings.BIKE_NAME, _settings.KIN8NM_NAME, _settings.YACHT_NAME, 
                  _settings.ENERGY_NAME, _settings.HOUSING_NAME, _settings.CONCRETE_NAME]
qm8_datasets = ['%s_%d'%(_settings.QM8_NAME, i) for i in range(16)]
qm9_datasets = ['%s_%d'%(_settings.QM9_NAME, i) for i in range(12)]
datasets = []
datasets += small_datasets
datasets += qm8_datasets
datasets += qm9_datasets
#datasets += ['UCI_Yacht', 'QM8_0', 'QM9_0']


#Rename dataset for QM
Y_NAME_MAPS = {_settings.QM8_NAME: dld.QM8Dataset.Y_NAMES, _settings.QM9_NAME: dld.QM9Dataset.Y_NAMES}
def dataset_rename(x):
    if 'QM8' in x:
        return "{}({})".format(x.split("_")[0], Y_NAME_MAPS[x.split("_")[0]][int(x.split("_")[1])])
    if 'QM9' in x:
        return "{}({})".format(x.split("_")[0], Y_NAME_MAPS[x.split("_")[0]][int(x.split("_")[1])])
    return x.replace("UCI_", "").replace("Boston", "").replace("Sharing", "")

In [4]:
#Define the function to use to read results (lo, hi, yhat, y, etc.)
def get_df_func_qm(dataset, method, seed, alpha=0.1):
    dataset, which_y = dataset.split("_")
    which_y = int(which_y)
    datakwargs_ = {'seed': seed, 'which_y': which_y}# utils.merge_dict_inline(datakwargs, )
    if method in conf_baselines:
        return demo.conformal_baselines_cached(method, dataset=dataset, datakwargs=datakwargs_, alpha=alpha)
    if method == 'DE':
        import models.baselines.DE as DE
        return DE.get_DE_result_cached(dataset, which_y, seed=seed, alpha=alpha)
    
    #===LVD
    if method == METHOD_NAME: method = '%s_NN_S_'%METHOD_NAME
    if method.startswith(METHOD_NAME):
        datakwargs_ = utils.merge_dict_inline({'test_split': dld.TEST, 'split_ratio': (60, 20, 20)}, datakwargs_)
        PIkwargs = {"kernel": 'trained', 'pred': {"KR": "KR", "NN": "base"}[method.split("_")[1]]}
        datakwargs_ = utils.merge_dict_inline(datakwargs_, {"train_split": dld.TRAIN, "val_split": dld.VALID})
        fitkwargs = utils.merge_dict_inline(default_fitkwargs, {"seed": seed, "ybar_bias": {"S":True, "NS":False}[method.split("_")[2]]})
        PI_model = {"MAD": 'LocalConformalMAD', "": "LocalConformal"}[method.split("_")[3]]
        return demo.eval_exp_cached(dataset, datakwargs_, regmodel._KERNEL_MLKR, fitkwargs, PI_model=PI_model, PIkwargs=PIkwargs, alpha=alpha)
    if method in baselines: return None
    raise NotImplementedError()
    

def get_df_func_other(dataset, method, seed, alpha):
    datakwargs = {'model_setting': 0, 'train_split': dld.TRAIN, 'val_split': dld.VALID, 'test_split': dld.TEST}
    
    #===LVD
    if method == METHOD_NAME: method = '%s_NN_S_'%METHOD_NAME
    if method.startswith(METHOD_NAME): #variants
        datakwargs_ = utils.merge_dict_inline(datakwargs, {'seed': seed})
        PIkwargs = {"kernel": 'trained', 'pred': {"KR": "KR", "NN": "base"}[method.split("_")[1]]}
        datakwargs_ = utils.merge_dict_inline(datakwargs_, {"train_split": dld.TRAIN, "val_split": dld.VALID})
        fitkwargs = utils.merge_dict_inline(default_fitkwargs, {"seed": seed, "ybar_bias": {"S":True, "NS":False}[method.split("_")[2]]})
        PI_model = {"MAD": 'LocalConformalMAD', "": "LocalConformal"}[method.split("_")[3]]
        return demo.eval_exp_cached(dataset, datakwargs_, regmodel._KERNEL_MLKR, fitkwargs, PI_model=PI_model, PIkwargs=PIkwargs, alpha=alpha)
    if method in conf_baselines:
        datakwargs_ = utils.merge_dict_inline(datakwargs, {'seed': seed})
        conf_fitkwargs = {'abs_resid': True} if method == 'LocalMAD' else {}
        return demo.conformal_baselines_cached(method, dataset, datakwargs_, 0, alpha=alpha, **conf_fitkwargs)
    if method in baselines:
        params = dict({"activation": 'ReLU', "num_hidden": 100, "num_layers": 1})
        train_params = dict({"num_iter": 1000,  "learning_rate": 1e-3})
        res = exp.run_experiments([method], [dataset], N_exp=[seed], damp=1e-2,  
                                  mode='exact', #'exact' 
                                  coverage=1-alpha, 
                                  params=params, train_params=train_params,
                                  data_path=DATA_PATH, cache_path = CACHE_PATH, quiet=True)
        return res[method][dataset][seed]
    raise NotImplementedError()

    
def get_df_func(dataset, method, seed, alpha):
    if _settings.QM8_NAME in dataset or _settings.QM9_NAME in dataset:
        return get_df_func_qm(dataset, method, seed, alpha)
    return get_df_func_other(dataset, method, seed, alpha)

def clean_df(summ_res, key, 
             index_rename=dataset_rename,
             column_rename=method_rename):
    df = summ_res[key]
    df.name = key
    df = df.reindex(datasets)
    df = df.rename(index=index_rename, columns=column_rename)
    return df
    

In [5]:
all_res = eval_utils._get_all_dfs(datasets, all_methods, range(NSEEDS), get_df_func)

reading all results: 100%|██████████| 4760/4760 [00:05<00:00, 845.54it/s] 


In [6]:
summ_res, _ = eval_utils.summary_key_metrics_all_datasets_and_methods(datasets, use_methods, None, None, all_res=all_res, width_methods=width_methods)

 30%|███       | 3/10 [00:00<00:00, 20.95it/s]

Aggregating results on UCI_BikeSharing


 30%|███       | 3/10 [00:00<00:00, 22.20it/s]

Aggregating results on Kin8nm


 40%|████      | 4/10 [00:00<00:00, 32.81it/s]

...

 10%|█         | 1/10 [00:00<00:00,  9.07it/s]

Aggregating results on QM9_10


 10%|█         | 1/10 [00:00<00:00,  9.13it/s]

Aggregating results on QM9_11


100%|██████████| 10/10 [00:02<00:00,  3.51it/s]


In [7]:
clean_df(summ_res, 'AUROC')

method,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,0.6816(0.1104),0.7173(0.0851),0.8475(0.3346),0.4575(0.0622)[2.26e-01<PBP],0.8615(0.1246),0.9432(0.0101)[6.83e-02>DE],0.4833(0.0099)
Kin8nm,0.6032(0.0105),0.6042(0.0187)[8.85e-01>LVD],0.6004(0.0208),0.4928(0.0224)[2.65e-01<DE],0.5053(0.0261),0.5410(0.0248),0.5363(0.0475)
Yacht,0.8350(0.0580),0.7774(0.0900),0.8493(0.0458)[5.48e-01>LVD],0.5239(0.0895),0.6084(0.0634),0.4703(0.0832)[1.83e-01<DJ],0.8078(0.0478)
Energy,0.7346(0.0627)[8.24e-01>MADSplit],0.7286(0.0557),0.7210(0.0825),0.5746(0.0809),0.5612(0.1099),0.5458(0.0551),0.4819(0.0262)[5.68e-03<MCDP]
Housing,0.5917(0.0845),0.6202(0.0835),0.6250(0.0667)[8.89e-01>MADSplit],0.4958(0.0585),0.6003(0.0681),0.4251(0.0776)[9.13e-02<PBP],0.4745(0.0363)
Concrete,0.6400(0.0606),0.6382(0.0571),0.6595(0.0711)[5.17e-01>LVD],0.4620(0.0490)[9.52e-02<PBP],0.5592(0.0622),0.5191(0.0349),0.4968(0.0385)
QM8(E1-CC2),0.6294(0.0137),0.6766(0.0083),0.5787(0.0380)[2.10e-03<LVD],NaN,0.9269(0.1394)[2.98e-04>MADSplit],NaN,NaN
QM8(E2-CC2),0.6168(0.0123),0.6623(0.0101),0.5684(0.0390)[3.34e-03<LVD],NaN,0.9181(0.1902)[2.12e-03>MADSplit],NaN,NaN
QM8(f1-CC2),0.8010(0.0141),0.8004(0.0104),0.7050(0.0290)[7.47e-07<MADSplit],NaN,0.9050(0.1446)[4.92e-02>LVD],NaN,NaN
QM8(f2-CC2),0.8234(0.0052),0.8093(0.0106),0.7855(0.0567)[2.22e-01<MADSplit],NaN,0.9556(0.0413)[2.72e-06>LVD],NaN,NaN


In [8]:
clean_df(summ_res, 'MAD')

method,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,0.0365(0.0308)[1.94e-01<DJ],0.0365(0.0308)[1.94e-01<DJ],7.3419(3.5139),0.0535(0.0251),3.3745(2.8968),124.5737(2.6790),162.2126(2.5775)[2.65e-17>MCDP]
Kin8nm,0.0712(0.0038)[1.13e-02<CQR],0.0712(0.0038)[1.13e-02<CQR],0.0772(0.0055),0.0919(0.0057),0.1877(0.0057),0.1815(0.0036),0.2221(0.1162)[3.74e-01>DE]
Yacht,1.8957(0.4797),1.8957(0.4797),3.5547(0.8493),10.1457(0.8377),11.2545(0.8091)[3.42e-01>MCDP],10.9172(0.7348),1.7964(0.2986)[5.87e-01<MADSplit]
Energy,2.9936(0.7536)[4.96e-01<DJ],2.9936(0.7536)[4.96e-01<DJ],3.4401(1.0353),3.1943(0.5121),3.7899(0.3075),8.1234(0.5918),11.6580(2.2411)[6.53e-04>MCDP]
Housing,3.3111(0.5334)[5.19e-01<CQR],3.3111(0.5334)[5.19e-01<CQR],3.4424(0.3318),3.6867(0.3250),4.4155(0.3923),6.0395(0.5445),7.9353(1.9739)[1.46e-02>MCDP]
Concrete,5.4391(0.5343)[5.85e-01<DJ],5.4391(0.5343)[5.85e-01<DJ],6.2057(1.0491),5.5781(0.5812),7.2203(0.7646),13.7490(0.6852),20.5883(3.5651)[1.61e-04>MCDP]
QM8(E1-CC2),0.0062(0.0002)[3.10e-07<CQR],0.0062(0.0002)[3.10e-07<CQR],0.0226(0.0039),NaN,4.7801(6.7969)[5.41e-02>CQR],NaN,NaN
QM8(E2-CC2),0.0074(0.0001)[1.21e-06<CQR],0.0074(0.0001)[1.21e-06<CQR],0.0214(0.0039),NaN,1.0067(0.6176)[6.95e-04>CQR],NaN,NaN
QM8(f1-CC2),0.0134(0.0004)[8.65e-10<CQR],0.0134(0.0004)[8.65e-10<CQR],0.0344(0.0027),NaN,1.9065(1.8629)[1.12e-02>CQR],NaN,NaN
QM8(f2-CC2),0.0287(0.0006)[9.83e-08<CQR],0.0287(0.0006)[9.83e-08<CQR],0.0640(0.0074),NaN,1.8516(2.3086)[3.68e-02>CQR],NaN,NaN


In [9]:
clean_df(summ_res, 'MCR')

method,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,90.4488(0.8274),89.8648(0.5855),89.8475(0.6691),95.2273(0.5711),100.0000(0.0000)[7.69e-10>DJ],71.8728(0.6793),0.6300(0.2000)[2.39e-22<MCDP]
Kin8nm,97.9853(0.5989),89.9512(0.7945)[5.36e-01<CQR],90.1526(0.6200),94.7070(0.3978),99.9573(0.0912),100.0000(0.0000)[1.73e-01>DE],100.0000(0.0000)[1.73e-01>DE]
Yacht,96.7742(2.1505)[7.68e-02>DJ],82.4194(7.0694),91.4516(4.6901),95.0000(2.0753),22.7419(5.9602)[6.51e-10<PBP],87.4194(4.1506),80.1613(10.8077)
Energy,93.9610(1.5920),90.2597(2.5056),90.3247(2.2169),96.1688(1.7967),97.9870(2.7112),100.0000(0.0000)[4.35e-02>DE],7.2078(5.8757)[1.98e-14<MADSplit]
Housing,96.8317(2.9445),90.5941(3.5315),91.6832(3.3397),97.6238(1.4157),96.0396(1.9244),100.0000(0.0000)[4.89e-04>DJ],8.1188(4.2726)[8.93e-20<MADSplit]
Concrete,97.4272(1.3156),88.8350(2.8306),88.5437(2.2561),98.0097(1.5763),97.8155(1.0298),100.0000(0.0000)[3.14e-03>DJ],3.3495(0.8392)[9.35e-19<CQR]
QM8(E1-CC2),92.4558(0.5402),90.0390(0.4530)[7.87e-01<CQR],90.1102(0.6794),NaN,100.0000(0.0000)[7.82e-12>LVD],NaN,NaN
QM8(E2-CC2),92.2447(0.5121),89.7705(0.6036)[2.68e-01<CQR],90.0574(0.5144),NaN,100.0000(0.0000)[3.78e-12>LVD],NaN,NaN
QM8(f1-CC2),92.5040(1.0869),90.1469(0.7227)[8.06e-01<CQR],90.2249(0.6776),NaN,100.0000(0.0000)[4.23e-09>LVD],NaN,NaN
QM8(f2-CC2),93.5644(0.6115),89.9151(0.6652),89.8784(0.7225)[9.07e-01<MADSplit],NaN,100.0000(0.0000)[9.83e-11>LVD],NaN,NaN


In [10]:
clean_df(summ_res, 'TCR')

method,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,90.2446(1.7183),89.2086(3.4559),58.6763(7.2667),85.5683(3.3076),100.0000(0.0000)[2.35e-08>LVD],49.9281(0.0000),0.0000(0.0000)[0.00e+00<MCDP]
Kin8nm,97.1865(1.5845),86.3609(2.5553),85.1682(2.2391)[2.82e-01<MADSplit],88.1346(1.7936),99.8777(0.2954),100.0000(0.0000)[2.23e-01>DE],100.0000(0.0000)[2.23e-01>DE]
Yacht,98.4615(3.2434)[3.84e-04>CQR],65.3846(23.8475),77.6923(12.2702),76.1538(9.8976),1.5385(4.8650)[2.53e-09<MCDP],50.0000(9.7638),70.0000(14.2533)
Energy,86.7742(5.7805),78.3871(10.8650),73.5484(11.9544),90.0000(6.5318),95.8065(6.2791),100.0000(0.0000)[6.39e-02>DE],9.6774(12.7228)[9.49e-10<CQR]
Housing,96.1905(4.3759),87.6190(8.7518),82.8571(8.1557),90.0000(5.2405),81.9048(9.7331),100.0000(0.0000)[2.24e-02>LVD],0.9524(3.0117)[7.27e-11<DE]
Concrete,97.0732(3.4108),83.9024(7.2899),85.3659(6.1917),95.6098(3.5993),91.7073(4.7684),100.0000(0.0000)[2.39e-02>LVD],3.4146(5.6561)[1.58e-15<MADSplit]
QM8(E1-CC2),90.1722(1.3193),88.1171(1.5320),83.5362(6.3979)[5.22e-02<MADSplit],NaN,100.0000(0.0000)[2.13e-09>LVD],NaN,NaN
QM8(E2-CC2),90.0804(1.6701),87.3938(2.1796),84.4317(4.3753)[7.72e-02<MADSplit],NaN,100.0000(0.0000)[1.58e-08>LVD],NaN,NaN
QM8(f1-CC2),90.4707(1.8250),85.0287(1.7681),80.0918(2.8378)[2.99e-04<MADSplit],NaN,100.0000(0.0000)[4.89e-08>LVD],NaN,NaN
QM8(f2-CC2),91.4351(1.5204),84.8565(2.1256),73.2836(4.7803)[1.20e-05<MADSplit],NaN,100.0000(0.0000)[2.51e-08>LVD],NaN,NaN


In [11]:
tdf = clean_df(summ_res, 'Width 50')
cols = tdf.columns
tdf['\\# finite'] = clean_df(summ_res, 'Count 50')['CQR'].map(lambda s: s.split("[")[0])
tdf = tdf.reindex(columns=['\\# finite'] + list(cols))
tdf

method,\# finite,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,3475.2000(1.2293),0.0569(0.0465)[6.93e-01<MADSplit],0.0658(0.0525),5.6182(4.4086),0.1445(0.0692),13.6236(6.6087),115.4807(0.8319)[1.79e-12>DE],0.8391(0.2672)
Kin8nm,1610.1000(10.1811),0.1414(0.0095),0.1220(0.0079),0.1191(0.0074)[3.99e-01<MADSplit],0.2538(0.0171),0.7957(0.0305),0.9818(0.0232),1.2856(0.1446)[8.25e-05>MCDP]
Yacht,61.9000(0.3162),3.9859(0.7947),3.1672(0.8371),2.8235(0.7430),19.2076(1.2717)[6.14e-08>MCDP],5.2379(0.7282),14.5240(0.7535),2.2873(0.4753)[7.34e-02<CQR]
Energy,154.0000(0.0000),6.0601(1.4139),5.7691(1.3706),5.1806(1.4710),9.8294(1.3942),10.5225(1.5916),30.1391(0.2441)[1.09e-11>DE],0.7818(0.0350)[5.65e-06<CQR]
Housing,98.3000(3.0569),6.6953(0.9707),6.0231(1.2310),4.9784(0.7167),11.5020(1.4020),10.2159(1.5205),30.6397(0.3464)[1.17e-12>DJ],0.7448(0.0795)[1.28e-08<CQR]
Concrete,200.1000(4.0125),10.9223(1.8452),9.7679(1.6624),9.3519(2.8447),48.0092(83.9986)[9.94e-01>MCDP],18.3001(2.9777),47.8162(0.3060),0.7773(0.0651)[5.30e-06<CQR]
QM8(E1-CC2),4320.8000(16.4033),0.0100(0.0004)[1.76e-03<MADSplit],0.0107(0.0004),0.0325(0.0045),NaN,18.6241(12.2653)[9.83e-04>CQR],NaN,NaN
QM8(E2-CC2),4324.5000(9.2766),0.0119(0.0003)[1.45e-02<MADSplit],0.0123(0.0004),0.0288(0.0032),NaN,13.3435(3.2480)[3.98e-07>CQR],NaN,NaN
QM8(f1-CC2),4305.6000(25.1802),0.0201(0.0009),0.0200(0.0012)[9.05e-01<LVD],0.0318(0.0049),NaN,14.7774(3.5689)[3.72e-07>CQR],NaN,NaN
QM8(f2-CC2),4276.6000(29.4437),0.0483(0.0024),0.0452(0.0016)[3.45e-03<LVD],0.0592(0.0126),NaN,15.3420(7.9987)[1.92e-04>CQR],NaN,NaN


In [12]:
tdf = clean_df(summ_res, 'Width 90')
cols = tdf.columns
tdf['\\# finite'] = clean_df(summ_res, 'Count 90')['CQR'].map(lambda s: s.split("[")[0])
tdf = tdf.reindex(columns=['\\# finite'] + list(cols))
tdf

method,\# finite,\methodname,MADSplit,CQR,DJ,DE,MCDP,PBP
Bike,3467.5000(4.4033),0.1508(0.1307)[5.32e-01<MADSplit],0.1856(0.1125),33.6500(21.5187),0.3224(0.1793),38.3135(13.2125),281.1955(1.6033)[3.65e-13>DE],2.0446(0.6521)
Kin8nm,938.0000(123.7192),0.3445(0.0225),0.2776(0.0203)[7.29e-01<CQR],0.2805(0.0157),0.4814(0.0303),1.8372(0.1578),2.4092(0.0551),2.9874(0.3588)[6.11e-04>MCDP]
Yacht,40.9000(2.8460),3.4652(1.3569),3.2931(1.0363)[7.54e-01<LVD],4.5190(2.0818),73.1358(1.7479)[2.36e-20>MCDP],13.1151(1.2231),34.2592(1.8803),3.4994(0.7305)
Energy,145.1000(11.0499),12.8940(2.0242),12.1864(2.7136),13.7574(2.8043),18.5538(2.1227),25.7091(5.6227),73.4684(0.7730)[4.00e-10>DE],1.9070(0.0852)[7.71e-07<MADSplit]
Housing,69.0000(19.1137),15.9447(2.6224),16.8129(7.4401),13.6962(1.8373),26.2053(1.8637),24.1735(1.8087),75.8974(1.4315)[6.61e-22>DJ],1.8148(0.1887)[5.90e-09<CQR]
Concrete,133.8000(20.1318),27.9349(3.5913),21.7933(2.9322),22.8666(3.4780),201.2672(370.0109)[4.87e-01>MCDP],43.4173(5.2261),116.4058(0.8270),1.8954(0.1630)[4.54e-09<MADSplit]
QM8(E1-CC2),4013.5000(142.8474),0.0279(0.0009),0.0263(0.0010)[1.49e-03<LVD],0.0936(0.0123),NaN,46.0402(30.5175)[1.03e-03>CQR],NaN,NaN
QM8(E2-CC2),4056.2000(71.6733),0.0329(0.0006),0.0304(0.0006)[2.41e-08<LVD],0.0927(0.0133),NaN,32.8494(7.9820)[3.94e-07>CQR],NaN,NaN
QM8(f1-CC2),3995.3000(140.3346),0.0520(0.0030)[2.87e-01<MADSplit],0.0549(0.0074),0.1056(0.0084),NaN,36.5003(8.7902)[3.65e-07>CQR],NaN,NaN
QM8(f2-CC2),3818.0000(148.5164),0.1096(0.0045),0.1069(0.0072)[3.30e-01<LVD],0.1770(0.0241),NaN,38.3440(20.7832)[2.57e-04>CQR],NaN,NaN
